In [ ]:
#!/usr/bin/env python
# coding: utf-8
from pyrosm import OSM, get_data
import osmnx as ox
import time
import networkx as nx
from shapely.geometry import Point, LineString, Polygon
import numpy as np
import geopandas as gpd
import pandas as pd

def nodes_to_linestring(path):
    coords_list = [(G.nodes[i]['x'], G.nodes[i]['y']) for i in path ]
    #print(coords_list)
    line = LineString(coords_list)
    
    return(line)

def shortestpath(x):
    o_lat, o_long, d_lat, d_long = x['o_lat'], x['o_long'], x['d_lat'], x['d_long']
    nearestnode_origin, dist_o_to_onode = ox.distance.get_nearest_node(G, (o_lat, o_long), method='haversine', return_dist=True)
    nearestnode_dest, dist_d_to_dnode = ox.distance.get_nearest_node(G, (d_lat, d_long), method='haversine', return_dist=True)
    
    #Add up distance to nodes from both o and d ends. This is the distance that's not covered by the network
    dist_to_network = dist_o_to_onode + dist_d_to_dnode
    shortest_p = nx.shortest_path_length(G,
                                      nearestnode_origin, 
                                      nearestnode_dest,
                                      weight='length') 
    
    #Total length to be covered is length along network between the nodes plus the distance from the O,D points to their nearest nodes
    total_length = shortest_p + dist_to_network
    intersec = 0
    path = nx.shortest_path(G,
                          nearestnode_origin, 
                          nearestnode_dest,
                          weight='length')
    intersections = ox.utils_graph.count_streets_per_node(G, path)
    for i in intersections:
        if intersections[i] >2:
            intersec += 1
    x["tc_length"] = total_length
    x["intersections"] = intersec
    return x, path
import taxicab as tc 
def shortestpath2(x):
    o_lat, o_long, d_lat, d_long = x['o_lat'], x['o_long'], x['d_lat'], x['d_long']
    route = tc.distance.shortest_path(G, (o_lat, o_long), (d_lat, d_long))  
    intersections = ox.utils_graph.count_streets_per_node(G, route[1])
    inter = 0
    for i in intersections:
        if intersections[i] >2:
            inter += 1
    x["tc_length"] = route[0]
    x["intersections"] = inter
    return x
import pickle as pickle 
with open('data_with_coord_29_04_21', "rb") as fh:
    df = pickle.load(fh)
df = df[(df.AttendanceTimeSeconds >0) & (df.dist_euclidian>100)]
df['velocity'] = df.dist_euclidian / df.AttendanceTimeSeconds
df = df[(df.CalYear>2016) & (df.velocity <50) & (df.velocity > 7)]
fragment, taille = 0, 1
print(df.shape)
df = df.iloc[0, :]
start = time.time()
# Initialize the reader
od_x, od_y = np.array((df.loc[['d_long', 'o_long']])), np.array((df.loc[['d_lat', 'o_lat']]))
north, south, east, west = od_y.max()+0.001, od_y.min()-0.001, od_x.max()+0.001, od_x.min()-0.001
print(north, south, east, west)

# Initiliaze with bounding box
osm = OSM(get_data("Greater London", directory="../pbf_data"),
          bounding_box=[west, south, east, north])
print(osm)
nodes, edges = osm.get_network(network_type='driving', nodes=True)
end1 = time.time()
print("node ok : ", end1 - start," s")
G = osm.to_graph(nodes, edges, graph_type="networkx")
end2 = time.time()
print("network ok : ", end2 - start," s")
# Plot the graph with OSMnx
#ox.plot_graph(G)
end3 = time.time()
print("la carte a charge en : ",end3 - start," s")

df['tc_length'], df['intersections'] = 0, 0
#from pandarallel import pandarallel
#df = df.progress_apply(myfunc, axis=1)
#pandarallel.initialize(progress_bar=True, nb_workers=20)
#df['osmnx_length'] = df.progress_apply(shortestpath, axis=1)
#df = df.parallel_apply(shortestpath2, axis=1)
#df = df.parallel_apply(shortestpath, axis=1)

df, path = shortestpath(df)
end4 = time.time()

print("Process total PBF: ",end4 - start," s")
#df.to_pickle('res/data_sample_osm_'+args["iter"])

In [ ]:
noeuds = pd.DataFrame({'lon':[], 'lat':[]})
for i in range(len(path)):
    noeuds = noeuds.append(nodes[nodes.id == path[i]][['lon','lat']])
noeuds

In [ ]:
from bokeh.plotting import figure
from bokeh.io import  push_notebook,output_notebook, show
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.tile_providers import get_provider
from bokeh.models.tools import HoverTool

# limites de la carte, +- 1 km
xutm_min, xutm_max, yutm_min, yutm_max = df.x_utm.min()-20000, df.x_utm.max()+20000, \
                                         df.y_utm.min()-20000, df.y_utm.max()+20000
# Tuile pour le fond de carte
tuile = get_provider('STAMEN_TERRAIN_RETINA')
# Limites de la carte
x_range, y_range, x_axis_type, y_axis_type = (xutm_min, xutm_max), \
                                             (yutm_min, yutm_max), \
                                             'mercator', 'mercator'
# Figure et fond de carte
p = figure(plot_width=1200, plot_height=1200, 
           x_range=x_range, y_range=y_range, 
           x_axis_type=x_axis_type, y_axis_type=y_axis_type)
p.add_tile(tuile)


In [ ]:
import compute_path as cp
def shortestpath(x):
    o_lat, o_long, d_lat, d_long = x['o_lat'], x['o_long'], x['d_lat'], x['d_long']
    nearestnode_origin, dist_o_to_onode = ox.distance.get_nearest_node(G, (o_lat, o_long), method='haversine', return_dist=True)
    nearestnode_dest, dist_d_to_dnode = ox.distance.get_nearest_node(G, (d_lat, d_long), method='haversine', return_dist=True)
    
    #Add up distance to nodes from both o and d ends. This is the distance that's not covered by the network
    dist_to_network = dist_o_to_onode + dist_d_to_dnode
    shortest_p = nx.shortest_path_length(G,
                                      nearestnode_origin, 
                                      nearestnode_dest,
                                      weight='length') 
    
    #Total length to be covered is length along network between the nodes plus the distance from the O,D points to their nearest nodes
    total_length = shortest_p + dist_to_network
    intersec = 0
    path = nx.shortest_path(G,
                          nearestnode_origin, 
                          nearestnode_dest,
                          weight='length')
    intersections = ox.utils_graph.count_streets_per_node(G, path)
    for i in intersections:
        if intersections[i] >2:
            intersec += 1
    x["tc_length_ox"] = total_length
    x["intersections_ox"] = intersec
    return x, path

In [ ]:
start = time.time()
G = ox.graph_from_point((df.o_lat, 
                         df.o_long), dist=20000, network_type='drive')
df, path_ox = shortestpath(df)
end = time.time()
print("Process total OX: ",end - start," s")

In [ ]:
noeuds_osmnx = pd.DataFrame({'lon':[], 'lat':[]})
for i in range(len(path_ox)):
    noeuds_osmnx = noeuds_osmnx.append({'lon':G.nodes[path_ox[i]]['x'],
                                        'lat':G.nodes[path_ox[i]]['y']}, ignore_index=True)
noeuds_osmnx

In [ ]:
import compute_path as cp
x2,y2 = cp.wgs84_to_web_mercator(noeuds_osmnx.lat, noeuds_osmnx.lon)


In [ ]:
start = time.time()
with open('sg.pkl', "rb") as f:
    sg = pickle.load(f)
with open('nodes.pkl', "rb") as f:
    nodes = pickle.load(f)

# on récupère les distances de node et les segments, ça prend 2 min environ
#nodes, sg = cp.compute_nodes(sgs, dist_type='euclidean')
path, length = cp.compute_path(nodes, sg,
                       df.o_long, 
                       df.o_lat,
                       df.d_long, 
                       df.d_lat)
end = time.time()
print("Process total SHP: ",end - start," s")
linepath, intersections = cp.get_linepath(sg, path)
x3,y3 = cp.wgs84_to_web_mercator(linepath[:, 1], linepath[:, 0])


In [ ]:
from bokeh.plotting import figure
from bokeh.io import  push_notebook,output_notebook, show
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.tile_providers import get_provider
from bokeh.models.tools import HoverTool

x_mid, y_mid = (df.x_utm+df.o_x_utm)/2, (df.y_utm+df.o_y_utm)/2
# limites de la carte, +- 1 km
xutm_min, xutm_max, yutm_min, yutm_max = x_mid-8000, x_mid+8000, \
                                         y_mid-8000, y_mid+8000
# Tuile pour le fond de carte
tuile = get_provider('CARTODBPOSITRON_RETINA')
# Limites de la carte
x_range, y_range, x_axis_type, y_axis_type = (xutm_min, xutm_max), \
                                             (yutm_min, yutm_max), \
                                             'mercator', 'mercator'
# Figure et fond de carte
p = figure(plot_width=1200, plot_height=1200, 
           x_range=x_range, y_range=y_range, 
           x_axis_type=x_axis_type, y_axis_type=y_axis_type)
p.add_tile(tuile)

# vert, rouge, bleu, jaune, violet
colors = ['#23851a', '#852b1a', '#1a4385', '#c87017' ,'#78199e']

p.line(x=x3, y=y3, line_color=colors[3], line_width=3)
p.line(x=x2, y=y2, line_color=colors[-1], line_width=3)
x,y = cp.wgs84_to_web_mercator(noeuds.lat, noeuds.lon)
p.line(x=x, y=y, line_color=colors[2], line_width=3)
p.line(x=[df.o_x_utm, 
          df.x_utm], 
       y=[df.o_y_utm, 
          df.y_utm], line_color=colors[1], line_width=3)
p.circle(x=[df.o_x_utm, 
          df.x_utm], 
        y=[df.o_y_utm, 
          df.y_utm], line_color=colors[:2], fill_color=colors[:2], size=20)

#lab = LabelSet(x='x',y='y',text='Station',x_offset=0,y_offset=0, source=source, text_align="center")
#p.add_layout(lab)
texts = ['Methode : Calcul, Distance',
         'PBF : 53s, 11058.4m',
         'OSMNX : 302s, 11213.8m',
         'SHP : 2.65s, 187645m',
         'Euc. : 0s, 13422m']
texts.reverse()
texts_c = ['#000000',
            colors[2],
            colors[-1],
            colors[3],
            colors[1]]
texts_c.reverse()
labs = pd.DataFrame({'x':[x_mid for i in range(5)],
                     'y':[y_mid for i in range(5)],
                     'text':texts,
                     'text_c':texts_c,
                     'x_offset':[5 for i in range(5)],
                     'y_offset':[200+i*30 for i in range(5)]})
source = ColumnDataSource(labs)
source
lab_pbf = LabelSet(x='x',
                   y='y',
                   text='text',
                   x_offset='x_offset',
                   y_offset='y_offset', 
                   text_align="center", 
                   text_color='text_c',
                   text_font_size="25pt",
                   background_fill_color='grey', 
                   background_fill_alpha=0.0,
                   source=source)

p.add_layout(lab_pbf)

p.xaxis.major_label_text_font_size = '25px'
p.xaxis.axis_label = "Lon"
p.xaxis.axis_label_text_font_size = '25px'

p.yaxis.major_label_text_font_size = '25px'
p.yaxis.axis_label = "Lat"
p.yaxis.axis_label_text_font_size = '25px'



show(p)

In [ ]:
labs

In [ ]:
length